# Rendering 3D spherical PLUTO data

# Part 1: generate data
generating the cartesian data out of the original pluto data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.interpolate import interpn
from astropy import constants as c

Note: I required [this fork of pyPLUTO](https://gitlab.mpcdf.mpg.de/sdoetsch/pypluto.git) to work, because of changes in the standard library, see [this fix](https://gitlab.mpcdf.mpg.de/sdoetsch/pypluto/-/commit/7b8243c6073785d1486f1bd495a98082f09f95f3)

In [ ]:
import pyPLUTO as pp
au = c.au.cgs.value

In [ ]:
data = pp.pload.pload(10, datatype='flt', w_dir='/Users/birnstiel/Desktop/PLUTO_Planet/data/')

UNIT_DENSITY  = 3.604e-13 # (gr/cm^3)
UNIT_VELOCITY = 6.679e+05 # (cm/s)
UNIT_LENGTH   = 2.992e+14 # (cm)

r = data.x1 * UNIT_LENGTH
ri = data.x1r * UNIT_LENGTH
th = data.x2
thi = data.x2r
ph = data.x3
phi = data.x3r
rho = data.rho.copy() * UNIT_DENSITY

Determine planet position

In [ ]:
_rho = rho[:, len(th)//2:len(th)//2+2, :].mean(1)

ir_p = (_rho / _rho.mean(-1)[:, None]).max(-1).argmax()
ip_p = _rho[ir_p, :].argmax()

Add $\Omega_\mathsf{planet}$

In [ ]:
vr = data.vx1 * UNIT_VELOCITY
vt = data.vx2 * UNIT_VELOCITY
vp = data.vx3 * UNIT_VELOCITY

#omegaP = np.sqrt(c.G.cgs.value * c.M_sun.cgs.value / UNIT_LENGTH**3)
#omega = vp / r[:, None, None] + omegaP
#vp = omega * r[:, None, None]

In [ ]:
rhod0 = data.rho * data.tr1 * UNIT_DENSITY
rhod1 = data.rho * data.tr2 * UNIT_DENSITY
rhod = rhod0 + rhod1

Roll planet to $\phi=0$

In [ ]:
rhod0 = np.roll(rhod0, -ip_p, axis=2)
rhod1 = np.roll(rhod1, -ip_p, axis=2)
rhod = rhod0 + rhod1
vr = np.roll(vr, -ip_p, axis=2)
vt = np.roll(vt, -ip_p, axis=2)
vp = np.roll(vp, -ip_p, axis=2)

## Inspect the data

Here are the original data dimensions

In [ ]:
r.shape, th.shape, ph.shape

In [ ]:
rho.shape

In [ ]:
def convert_vel_pol2cart(r, theta, phi, vr, vt, vp):
    r     = r[:, None, None]
    theta = theta[None, :, None]
    phi   = phi[None, None, :]
    
    vx = vr * np.sin(theta) * np.cos(phi) + vt * np.cos(theta) * np.cos(phi) - vp * np.sin(theta) * np.sin(phi)
    vy = vr * np.sin(theta) * np.sin(phi) + vt * np.cos(theta) * np.sin(phi) + vp * np.sin(theta) * np.cos(phi)
    vz = vr * np.cos(theta) * vt * np.sin(phi)

    return vx, vy, vz

In [ ]:
vx, vy, vz = convert_vel_pol2cart(r, th, ph, vr, vt, vp)

make a 2D vertical slice

In [ ]:
rri, tti = np.meshgrid(ri, thi, indexing='ij')

xxi = rri * np.sin(tti)
zzi = rri * np.cos(tti)

i_phi = 0

f, ax = plt.subplots(dpi=200)
vmax = 10.**np.ceil(np.log10(rhod.max()))
cc =ax.pcolormesh(xxi / au, zzi / au, rhod[:, :, i_phi], norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax), shading='flat')
ax.set_aspect('equal')
pos = ax.get_position()
cax = f.add_axes([pos.x1, pos.y0, pos.height / 30, pos.height])
cb = plt.colorbar(cc, cax=cax)
cb.set_label('$\\rho_{dust}$')

Quiver plot

In [ ]:
rri, ppi = np.meshgrid(ri, phi, indexing='ij')
rr, pp = np.meshgrid(r, ph, indexing='ij')

xxi = rri * np.cos(phi)
yyi = rri * np.sin(phi)

xx = rr * np.cos(pp)
yy = rr * np.sin(pp)

i_th = rho.shape[1]//2

i_r_in = r.searchsorted(17.5 * au)
i_r_out = r.searchsorted(22.5 * au)

f, ax = plt.subplots(dpi=200)
vmax = 10.**np.ceil(np.log10(rhod.max()))
cc =ax.pcolormesh(
    xxi[i_r_in:i_r_out + 1, :] / au,
    yyi[i_r_in:i_r_out + 1, :] / au,
    rhod[i_r_in:i_r_out,:, :][:, i_th, :], norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax), shading='flat')
ax.set_aspect('equal')
pos = ax.get_position()
cax = f.add_axes([pos.x1, pos.y0, pos.height / 30, pos.height])
cb = plt.colorbar(cc, cax=cax)
cb.set_label('$\\rho_{dust}$')

s = 1

x_arr = xx[i_r_in:i_r_out,...][::s,::s] / au
y_arr = yy[i_r_in:i_r_out,...][::s,::s] / au
vx_arr = vx[i_r_in:i_r_out,...][::s, i_th, ::s] / au * 3.15e7
vy_arr = vy[i_r_in:i_r_out,...][::s, i_th, ::s] / au * 3.15e7

v = np.hypot(vx_arr, vy_arr)
vx_arr *= np.log10(v) / v
vy_arr *= np.log10(v) / v

ax.quiver(x_arr, y_arr, vx_arr, vy_arr, color='w', pivot='mid', scale=100)#, angles='uv')

_phi = 0
ax.set_xlim(20 * np.cos(np.deg2rad(_phi)) + np.array([-2.5, 2.5]))
ax.set_ylim(20 * np.sin(np.deg2rad(_phi)) + np.array([-2.5, 2.5]))

Divide out the mid-plane gradient. This normalizes the disk to mostly take out the general radial gradient.

In [ ]:
rho_mid = rho[:, data.n2//2].mean(-1)
rho_mid_ini = 4e-12 * (data.x1/data.x1[0])**-2.5

In [ ]:
f, ax = plt.subplots()
ax.loglog(data.x1, rho_mid)
ax.loglog(data.x1, rho_mid_ini)

In [ ]:
rri, tti = np.meshgrid(ri, thi, indexing='ij')

xxi = rri * np.sin(tti)
zzi = rri * np.cos(tti)

i_phi = 0

rho_norm = rhod * 100 / rho_mid_ini[:, None, None]

f, ax = plt.subplots(dpi=200)
vmax = 10.**np.ceil(np.log10(rho_norm.max()))
cc =ax.pcolormesh(xxi / au, zzi / au, rho_norm[:, :, i_phi], norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax), shading='flat')
ax.set_aspect('equal')
pos = ax.get_position()
cax = f.add_axes([pos.x1, pos.y0, pos.height / 30, pos.height])
cb = plt.colorbar(cc, cax=cax)
cb.set_label('$\\rho_{gas}$')

## Prepare for interpolation

select which density we want to write out, here we take the normalized dust density defined above.

In [ ]:
rho_out = rho_norm

also a bit tricky: we define the density only grid centers, so there is a gap in phi direction between the last and the first grid center as the interpolation will not know about the periodic direction. We close this here by adding another point beyond $2\pi$ which is a copy of the first point near $\phi = 0$.

In [ ]:
ph_mod = np.hstack((ph - ph[0], 2 * np.pi))
rho_mod = np.concatenate((rho_out, rho_out[:, :, 0:1]), axis=2)
vx_mod = np.concatenate((vx, vx[:, :, 0:1]), axis=2)
vy_mod = np.concatenate((vy, vy[:, :, 0:1]), axis=2)
vz_mod = np.concatenate((vz, vz[:, :, 0:1]), axis=2)

We create a cartesian slice, here in the mid-plane `(x, y)` while we call the height `z`

In [ ]:
dx = 0.08
#dx = 0.3

x = np.arange(-40, 40, dx) * au
y = np.arange(-40, 40, dx) * au
z = np.arange(-8, 8, dx) * au

X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

We translate the coordinates of that slice to spherical coordinates.

Note that phi goes from 0 to $2 \pi$ in the original data, but the output of `np.arctan2` has negative angles and needs to be shifted

In [ ]:
R = np.sqrt(X**2 + Y**2 + Z**2)
T = np.pi/2 - np.arctan2(Z, np.sqrt(X**2 + Y**2))
P = (np.arctan2(Y, X) + 2 * np.pi) % (2 * np.pi)

We create a array of new points, shape is (N, 3)

In [ ]:
points = np.array([R.ravel(), T.ravel(), P.ravel()]).T

We call the interpolation. There's values outside the box, so we assign them the value `0.0`.

In [ ]:
interp_rho = interpn((r, th, ph_mod), rho_mod, points, fill_value=0.0, bounds_error=False)
interp_vx = interpn((r, th, ph_mod), vx_mod, points, fill_value=0.0, bounds_error=False)
interp_vy = interpn((r, th, ph_mod), vy_mod, points, fill_value=0.0, bounds_error=False)
interp_vz = interpn((r, th, ph_mod), vz_mod, points, fill_value=0.0, bounds_error=False)

The result is again one dimensional (one value per new point), so we need to reshape it to match the shape of the slice. After this, it is again 3-dimensional, since our slice has 3 dimensions, but the z-dimension is just one value here.

In [ ]:
interp_rho = interp_rho.reshape(X.shape)
interp_vx = interp_vx.reshape(X.shape)
interp_vy = interp_vy.reshape(X.shape)
interp_vz = interp_vz.reshape(X.shape)

Plot it

In [ ]:
f, axs = plt.subplots(1, 2, dpi=200, gridspec_kw={'width_ratios':[4, 1]}, figsize=(10, 2))

vmax = 10.**np.ceil(np.log10(interp_rho.max()))

iy = len(y) // 2 
iz = len(z) // 2

ax = axs[0]
cc1 = ax.pcolormesh(X[:, iy, :] / au, Z[:, iy, :] / au, interp_rho[:, iy, :], norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax), shading='auto', rasterized=True)

ax = axs[1]
cc2 = ax.pcolormesh(X[:, :, iz] / au, Y[:, :, iz] / au, interp_rho[:, :, iz], norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax), shading='auto', rasterized=True)


for ax, cc in zip(axs, [cc1, cc2]):
    ax.set_aspect('equal')
    pos = ax.get_position()
    cax = f.add_axes([pos.x0, pos.y1, pos.width, pos.height / 20])
    cb = plt.colorbar(cc, cax=cax, orientation='horizontal')
    cb.set_label('$\\rho_{gas}$')
    cax.xaxis.set_label_position('top')
    cax.xaxis.set_ticks_position('top')

## Export data

store the data in a simple format

In [ ]:
np.savez('pluto_total_dust.npz', x=x, y=y, z=z, rho=interp_rho, vx=interp_vx, vy=interp_vy, vz=interp_vz)

this can be loaded with:

# Part 2: Rendering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from volrender import TransferFunction, Renderer, render_movie
from matplotlib.colors import LogNorm, Normalize

## Load interpolated data

In [ ]:
with np.load('pluto_total_dust.npz') as f:
    x = f['x']
    y = f['y']
    z = f['z']
    rho = f['rho']
    vx = f['vx']
    vy = f['vy']
    vz = f['vz']

## Rescale data and define the transfer function

In [ ]:
vmax = rho.max()
norm = LogNorm(vmin=1e-8 * vmax, vmax=vmax, clip=True)
datacube = norm(rho.ravel()).reshape(rho.shape)

**Create the renderer**  
Note: this is quite high resolution and takes of the order of a minute to render. Reduce `N` to something like 300 to get results in seconds for testing.

In [ ]:
tf = TransferFunction(x0=[0.15, 0.42, 0.7])
r = Renderer(datacube, tf=tf, N=800)

update the transfer function

In [ ]:
tf.sigma[:] = np.array([0.03, 0.02, 0.08])
tf.colors[:, -1] = np.array([0.05, 0.02, 0.8])

In [ ]:
tf.colors[:, :3] = np.array([
    [1.00, 0.00, 0.25],
    [0.25, 0.25, 0.75],
    [0.15 ,0.90, 0.15],
    ])

#tf.colors[:, -1] = np.array([0.05, 0.02, 1.0])
tf.colors[:, -1] = np.array([0.025, 0.01, 0.7])

In [ ]:
plt.imshow([tf.colors[:,:3]])

## Render images

In [ ]:
%%time
r.render(phi=40, theta=60, transparent=False, bg=1.0)

In [ ]:
f, ax = r.plot(diagnostics=True, cb_norm=norm, L=80, invert=True)

Make two plots, one with white background, the other one transparent.

**Note:** the alpha channel contains the optical depth, i.e. we have to invert it. To get a good looking scaling, we also take it to the fourth power.

In [ ]:
# alpha = 1 - r.image[:,:,-1]**4
alpha = 1 - (r.image.sum(-1)/r.image.sum(-1).max())**8
f, ax = plt.subplots(figsize=(10, 4), dpi=300)
ax.set_ylim(-12, 15)
_ = r.plot(alpha=alpha, cb_norm=norm, L=40, invert=True, ax=ax)

ax.set_facecolor('none')
ax.add_artist(plt.Circle((10, 0.5), 1.5, ec='k', fc='none', alpha=0.5, ls='--'))
ax.annotate('planet', (10, 2),
            xytext=(10, 7), horizontalalignment='center',
            arrowprops=dict(facecolor='black', width=.5, headwidth=5, headlength=5),
    )
ax.annotate('vortex', (-3, 8),
            xytext=(-3, 13), horizontalalignment='center',
            arrowprops=dict(facecolor='black', width=.5, headwidth=5, headlength=5),
    )
f.get_axes()[1].set_ylabel(r'$\rho/\rho_0$')

f.savefig(f'disk.pdf', dpi=300, transparent=True, bbox_inches='tight')

## Render movie

Again: N is quite high, so each frame takes about a minute to render on my laptop. Reduce this to get a (probably $N^3$) speed-up.

In [ ]:
phi = np.linspace(0, 360, 97)[:-1]
fname = '../output/movie_disk.mp4'
render_movie(datacube, theta=60 *np.ones_like(phi), phi=phi, ncpu=1, tf=tf, N=800, dpi=600, bg=1.0, fname=fname)

In [ ]:
from IPython.display import HTML
HTML(f"""
<video width="500" controls loop autoplay>
  <source src="{fname}" type="video/mp4">
</video>
""")

Try making a transparent movie:

In [ ]:
phi = np.linspace(0, 360, 97)[:-1]
fname = '../output/movie_disk.mov'
render_movie(datacube, theta=60 *np.ones_like(phi), phi=phi, ncpu=1, tf=tf, N=600, dpi=600, bg=1.0, fname=fname,
             alpha=lambda image: 1 - (image.sum(-1)/image.sum(-1).max())**8)

# Part 3: Line Integral Convolution

## Load interpolated data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from volrender import TransferFunction, Renderer, render_movie
from matplotlib.colors import LogNorm, Normalize

import volrender
import astropy.units as u
import astropy.constants as c

au = c.au.cgs.value

In [ ]:
with np.load('pluto_total_dust.npz') as f:
    x = f['x']
    y = f['y']
    z = f['z']
    rho = f['rho']
    vx = f['vx']
    vy = f['vy']
    vz = f['vz']

In [ ]:
nx, ny, nz = rho.shape
i_mid = nz // 2

## Azimuthal plot

### Quiver Plot
as sanity check, here r-phi mid-plane

In [ ]:
v = np.array([vx[..., i_mid],  vy[..., i_mid]]).transpose(1, 2, 0)
s = 10

f, ax = plt.subplots(dpi=100)
ax.set_aspect('equal')
ax.pcolormesh(x / au, y / au, rho[:, :, i_mid], norm=LogNorm())
ax.quiver(x[::s] / au, y[::s] / au, vx[::s, ::s, i_mid].T, vy[::s, ::s, i_mid].T, pivot='mid')

for _p in np.linspace(12,17,20):
    res = volrender.lic.calc_2D_streamline(np.array([(_p-1) * au, (_p+1) * au]), x, y, v, rho[..., i_mid], length=20*au, direction='both')
    ax.plot(*(res[0]/au).T, 'r-', lw=.5)

### LIC

Down-scaling parameter. 1 is full resolution (4 min), 2 means half resolution (few seconds).

In [ ]:
s = 1

Get the velocity in the right format

In [ ]:
vel = np.array([vx[::s, ::s, i_mid],  vy[::s, ::s, i_mid]]).transpose(1, 2, 0)
_x = x[::s]
_y = y[::s]

Call the LIC

In [ ]:
noise = volrender.lic.gen_noise_fast(*vel.shape[:-1])
%time lic1 = volrender.lic.LIC(noise, _x, _y, vel, length=min(_x[-1] - _x[0], _y[-1] - _y[0]) / 10.)
plt.imshow(lic1)

In [ ]:
%time lic2 = volrender.lic.LIC_twostage(_x, _y, vel, generate_plot=True)

Magic scaling to make it look good.

In [ ]:
f = 1
fig, axs = plt.subplots(2, 4, figsize=(8, 4), dpi=200)

for ilic, lic in enumerate([lic1, lic2]):

    vmax = rho[..., i_mid].max()
    axs[ilic, 0].imshow(rho[..., i_mid], norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax), cmap='magma', interpolation='none')

    axs[ilic, 1].imshow(lic, norm=Normalize(vmin=0, vmax=1), cmap='gray', interpolation='none')

    Q = 10**(-f + 2 * f * Normalize()(lic)) * rho[::s, ::s, i_mid]
    vmax = Q.max()
    axs[ilic, 2].imshow(Q, norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax), cmap='magma', interpolation='none')

    vmax = rho[..., i_mid].max()
    rgb = volrender.lic.hsv_mix(rho[::s, ::s, i_mid], lic, norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax))
    volrender.lic.pcolormesh_rgb(_x, _y, rgb, ax= axs[ilic, 3], rasterized=True)
    axs[ilic, 3].set_aspect('equal')

for ax in axs.ravel():
    ax.set_facecolor('none')
    ax.set_axis_off()
    
fig.savefig('comparison.pdf', transparent=True, bbox_inches='tight')

In [ ]:
for f in [0, 1,2,3,4]:
    fig, ax = plt.subplots(dpi=200)
    Q = 10**(-f + 2 * f * Normalize()(lic1)) * rho[::s, ::s, i_mid]
    vmax = Q.max()
    ax.imshow(Q, norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax), cmap='magma', interpolation='none')
    ax.set_axis_off()
    ax.set_facecolor('k')
    fig.savefig(f'/Users/birnstiel/Desktop/LIC_disk_f{f}.pdf', transparent=True, bbox_inches='tight')

## Vertical Plot

In [ ]:
s = 1
ir0 = x.searchsorted(7 * au)
i_slice = nx//2

In [ ]:
_x = x[ir0::s]
_y = z[::s]

_v = np.array([vx[ir0::s, i_slice, ::s],  vz[ir0::s, i_slice, ::s]]).transpose(1, 2, 0)
_rho = rho[ir0::s, i_slice, ::s]

vmax = _rho.max()

f, ax = plt.subplots(figsize=(10, 5))
ax.pcolormesh(_x / au, _y / au, _rho.T, norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax))
ax.set_aspect('equal')
ax.set_ylim(-8, 8)

In [ ]:
noise = volrender.lic.gen_noise_fast(*_v.shape[:-1], sigma=0.5)
%time lic1 = volrender.lic.LIC(noise, _x, _y, _v, length=min(_x[-1] - _x[0], _y[-1] - _y[0]) / 5.)

In [ ]:
%time lic2 = volrender.lic.LIC_twostage(_x, _y, _v, generate_plot=True)

In [ ]:
f, ax = plt.subplots(figsize=(10, 5))
ax.imshow(lic2.T)

In [ ]:
Q = _rho * 10.**(-1 + 2 * Normalize()(lic2))

f, ax = plt.subplots(dpi=200)
vmax = Q.max()
ax.pcolormesh(_x / au, _y / au, Q.T, rasterized=True, norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax), shading='gouraud')

In [ ]:
im = volrender.lic.hsv_mix(_rho, lic2, norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax))

In [ ]:
f, ax = plt.subplots(dpi=200)
volrender.lic.pcolormesh_rgb(_x / au, _y / au, im, rasterized=True, ax=ax)